In [1]:
from dataload import *

In [23]:
x_train,y_train = load_object('samplexy.dat')

In [19]:
x_dep, x_valid, y_dep, y_valid = load_object("reduce.dat")

In [3]:
x_test, y_test = load_object('correduce.dat')

In [25]:
x_train.shape

(149995, 35)

In [24]:
print(x_test.shape,y_test.shape,x_valid.shape,y_valid.shape)

(311029, 35) (311029, 2) (489844, 35) (489844, 2)


In [15]:
def score_test(model, x_test, y_test):
    from pandas import DataFrame
    y_pred = model.predict(x_test)
    # recode NaNs in y_test.attack_type
    y_recode = y_test.attack_type
    y_recode[y_recode.isna()] = 'unknown'
    labels = ['normal', 'dos', 'probe', 'r2l', 'u2r', 'unknown']
    return DataFrame({
        'precision' : precision_score(y_recode, y_pred, average=None, labels=labels),
        'recall' : recall_score(y_recode, y_pred, average=None, labels=labels),
        'F1' : f1_score(y_recode, y_pred, average=None, labels=labels) },
        # 'kappa' : cohen_kappa_score(y_test.attack_type, y_pred, labels=labels) },
        # 'AUC' : roc_auc_score(y_test.attack_type, y_pred, average=None) },
        index=labels)

# normal-vs-rest recoding of classes
def score_test2(model, x_test, y_test):
    from pandas import DataFrame
    y_pred = model.predict(x_test)
    # recode non-normal classes in y_test.attack_type
    y_recode = y_test.attack_type
    y_recode[y_recode.isna()] = 'unknown'
    y_recode[y_recode != 'normal'] = 'attack'
    y_pred[y_pred != 'normal'] = 'attack'
    labels = ['normal', 'attack']
    return DataFrame({
        'precision' : precision_score(y_recode, y_pred, average=None, labels=labels),
        'recall' : recall_score(y_recode, y_pred, average=None, labels=labels),
        'F1' : f1_score(y_recode, y_pred, average=None, labels=labels) },
        # 'AUC' : roc_auc_score(y_recode, y_pred, average=None) },
        index=labels)

from sklearn import metrics,linear_model,naive_bayes,svm, model_selection, tree, ensemble,datasets, feature_extraction
from sklearn.feature_selection import RFE,SelectKBest,f_regression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import log_loss, confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score
from sklearn.model_selection import KFold 
from sklearn.cross_validation import cross_val_score, cross_val_predict

D:\Programs\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [26]:
rforest = ensemble.RandomForestClassifier(
    n_estimators=50, 
    max_depth=20, random_state = 2018)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=True, random_state=2018, verbose=0, warm_start=False)

In [27]:
score_test2(rforest,x_valid,y_valid)

,F1,precision,recall
normal,0.999090,0.999825,0.998355
attack,0.999775,0.999593,0.999957


In [28]:
score_test(rforest,x_valid,y_valid)

D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,F1,precision,recall
normal,0.99909,0.999825,0.998355
dos,0.00000,0.000000,0.000000
probe,0.00000,0.000000,0.000000
r2l,0.00000,0.000000,0.000000
u2r,0.00000,0.000000,0.000000
unknown,0.00000,0.000000,0.000000


In [30]:
score_test(rforest,x_test,y_test)

D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,F1,precision,recall
normal,0.845796,0.736147,0.993828
dos,0.998697,0.998929,0.998464
probe,0.661373,0.498824,0.981069
r2l,0.310855,0.870299,0.189221
u2r,0.365591,0.314815,0.435897
unknown,0.000000,0.000000,0.000000


In [31]:
score_test2(rforest,x_test,y_test)

,F1,precision,recall
normal,0.845796,0.736147,0.993828
attack,0.954222,0.998368,0.913814


In [92]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as randint

In [95]:
clf = RandomForestClassifier(n_estimators=50)

In [79]:
type(random_search)

sklearn.model_selection._search.RandomizedSearchCV

In [80]:
random_search

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=5,
          param_distributions={'criterion': ['gini', 'entropy'], 'n_estimators': 39, 'max_depth': 24, 'max_features': 9, 'min_samples_split': 8, 'min_samples_leaf': 10, 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [93]:
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 11),
              "min_samples_split": randint(2, 11),
              "min_samples_leaf": randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

In [96]:
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

In [99]:
from time import time
start = time()
random_search.fit(x_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

RandomizedSearchCV took 178.29 seconds for 20 candidates parameter settings.


NameError: name 'report' is not defined

In [ ]:
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html

In [100]:
random_search.cv_results_

D:\Programs\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Programs\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Programs\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Programs\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:1

{'mean_fit_time': array([2.01461069, 0.77426227, 4.48367206, 1.97903903, 1.27525552,
        2.35303879, 3.67683053, 2.68814206, 4.49032092, 1.05251789,
        3.23933442, 0.68583186, 1.83642046, 1.83176637, 1.37565327,
        1.37000187, 4.28221138, 0.98735905, 4.11067001, 1.85902707]),
 'mean_score_time': array([0.15225919, 0.15491867, 0.19580944, 0.19846892, 0.20644744,
        0.1542538 , 0.19979874, 0.19348232, 0.19447947, 0.19314988,
        0.19049048, 0.15957308, 0.20013126, 0.15791059, 0.15691312,
        0.15990527, 0.18849532, 0.15691344, 0.20146092, 0.156581  ]),
 'mean_test_score': array([0.90631688, 0.91458382, 0.9990333 , 0.99921331, 0.99739325,
        0.90098337, 0.99933998, 0.99890663, 0.99933331, 0.99761992,
        0.99914664, 0.91980399, 0.99876663, 0.90245008, 0.90577686,
        0.90275676, 0.99929331, 0.89870329, 0.99933331, 0.90237008]),
 'mean_train_score': array([0.90620688, 0.91484042, 0.99943998, 0.99973332, 0.99785326,
        0.90104011, 0.9999    , 0.9

In [101]:
print('best score:', random_search.best_score_)
print('best parameters: ', random_search.best_params_)

best score: 0.9993399779992667
best parameters:  {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 6, 'min_samples_leaf': 1, 'min_samples_split': 4}


In [102]:
score_test(random_search,x_test,y_test)

D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,F1,precision,recall
normal,0.844826,0.734886,0.993448
dos,0.000000,0.000000,0.000000
probe,0.000000,0.000000,0.000000
r2l,0.000000,0.000000,0.000000
u2r,0.000000,0.000000,0.000000
unknown,0.000000,0.000000,0.000000


In [103]:
score_test2(random_search,x_test,y_test)

,F1,precision,recall
normal,0.844826,0.734886,0.993448
attack,0.953888,0.998267,0.913287


In [106]:
best_rf = ensemble.RandomForestClassifier(
    bootstrap=False,
    criterion="gini",
    max_features=6, 
    max_depth=None, random_state = 2018)


In [107]:
best_rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [108]:
score_test(best_rf,x_test,y_test)

D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,F1,precision,recall
normal,0.843838,0.732978,0.994207
dos,0.000000,0.000000,0.000000
probe,0.000000,0.000000,0.000000
r2l,0.000000,0.000000,0.000000
u2r,0.000000,0.000000,0.000000
unknown,0.000000,0.000000,0.000000


In [109]:
score_test2(best_rf,x_test,y_test)

,F1,precision,recall
normal,0.843838,0.732978,0.994207
attack,0.953478,0.998466,0.912369


In [111]:
score_test(best_rf,x_valid,y_valid)

D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,F1,precision,recall
normal,0.999265,0.999743,0.998787
dos,0.000000,0.000000,0.000000
probe,0.000000,0.000000,0.000000
r2l,0.000000,0.000000,0.000000
u2r,0.000000,0.000000,0.000000
unknown,0.000000,0.000000,0.000000


In [112]:
nic_rf = RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=19, max_features=9,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=9, min_weight_fraction_leaf=0.0,
            n_estimators=51, n_jobs=-1, oob_score=False, random_state=4129,
            verbose=0, warm_start=False)


In [113]:
nic_rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=19, max_features=9,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=9, min_weight_fraction_leaf=0.0,
            n_estimators=51, n_jobs=-1, oob_score=False, random_state=4129,
            verbose=0, warm_start=False)

In [114]:
score_test(nic_rf,x_valid,y_valid)

D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,F1,precision,recall
normal,0.999244,0.999825,0.998664
dos,0.000000,0.000000,0.000000
probe,0.000000,0.000000,0.000000
r2l,0.000000,0.000000,0.000000
u2r,0.000000,0.000000,0.000000
unknown,0.000000,0.000000,0.000000


In [115]:
score_test(nic_rf,x_test,y_test)

D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,F1,precision,recall
normal,0.844135,0.733688,0.993729
dos,0.000000,0.000000,0.000000
probe,0.000000,0.000000,0.000000
r2l,0.000000,0.000000,0.000000
u2r,0.000000,0.000000,0.000000
unknown,0.000000,0.000000,0.000000


In [118]:
param_grid_cfm = {"criterion" : ['gini', 'entropy'],
                    "n_estimators": [42, 44, 46],
                    "max_features": [8, 9, 10],
                    "max_depth": [20, 21, 22],
                    "min_samples_split": [2, 3, 4],
                    "min_samples_leaf": [1, 2, 3],
                    "bootstrap": [True, False]}

In [119]:
clf_cfm = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='entropy',n_jobs = 5, random_state=4129)

In [122]:
rf_cv = GridSearchCV(estimator = clf_cfm, param_grid = param_grid_cfm, 
                             cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, 
                                                                        random_state = 2018), n_jobs = 5)

In [123]:
%time rf_cv.fit(x_train,y_train)

Wall time: 50min 28s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=2018, shuffle=True),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=5, oob_score=False, random_state=4129,
            verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=5,
       param_grid={'criterion': ['gini', 'entropy'], 'n_estimators': [42, 44, 46], 'max_features': [8, 9, 10], 'max_depth': [20, 21, 22], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2, 3], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [125]:
rf_cv.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 20,
 'max_features': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 44}

In [131]:
grid_rf = RandomForestClassifier(bootstrap=False, class_weight="balanced",
            criterion='entropy', max_depth=20, max_features=10,
            min_samples_leaf=1, min_samples_split=4,
            n_estimators=44, n_jobs=4, random_state=4129,
            verbose=0, warm_start=False)

In [132]:
%time grid_rf.fit(x_train,y_train)

Wall time: 2.66 s


RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='entropy', max_depth=20, max_features=10,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=4, min_weight_fraction_leaf=0.0,
            n_estimators=44, n_jobs=4, oob_score=False, random_state=4129,
            verbose=0, warm_start=False)

In [133]:
score_test(grid_rf,x_test,y_test)

D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,F1,precision,recall
normal,0.843787,0.733109,0.993828
dos,0.000000,0.000000,0.000000
probe,0.000000,0.000000,0.000000
r2l,0.000000,0.000000,0.000000
u2r,0.000000,0.000000,0.000000
unknown,0.000000,0.000000,0.000000


In [134]:
score_test2(grid_rf,x_test,y_test)

,F1,precision,recall
normal,0.843787,0.733109,0.993828
attack,0.953482,0.998366,0.912461
